# SETUP

In [49]:
# Read libraries
import pandas as pd
import numpy as np
import os
import re
import networkx as nx
from tqdm import tqdm
import matplotlib.pyplot as plt



# ML libraries
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from langdetect import detect, DetectorFactory
import torch

from ortools.graph.python import min_cost_flow

# Open file in read mode

taxonomy_path = os.path.join("data", "taxonomy.txt")
count_of_products_per_level1_path = os.path.join("data", "count_of_products_per_level1.csv")
data_path = os.path.join("data", "ensae_export_without_l1.parquet")

## Read categories files

In [2]:
# --------- Lire la taxonomy depuis un fichier txt ----------
# Assumons que le fichier s'appelle "taxonomy.txt"
# Format attendu : id_path <tab> category_path
df_taxonomy = pd.read_csv(taxonomy_path, sep='\t', header=None, names=['id_path', 'category_path'])

# Nettoyage
df_taxonomy['category_path'] = df_taxonomy['category_path'].str.strip()
df_taxonomy['id_path'] = df_taxonomy['id_path'].str.strip()

# Construire le graphe dirigé de la taxonomie
G = nx.DiGraph()
root = "ROOT"  # racine commune
G.add_node(root)

for path in df_taxonomy['category_path']:
    parts = [p.strip() for p in path.split(">")]
    if parts:  # relier le level_1 à la racine
        G.add_edge(root, parts[0])
    for i in range(len(parts)-1):
        parent = parts[i]
        child = parts[i+1]
        G.add_edge(parent, child)

# Identifier les level_1
level_1_nodes = [p.split(">")[0].strip() for p in df_taxonomy['category_path']]
level_1_nodes = list(set(level_1_nodes))

print("Level 1 categories:", level_1_nodes)

# Créer un DataFrame par level_1 avec la concaténation de tous ses descendants
level1_texts = {}
for lvl1 in level_1_nodes:
    descendants = nx.descendants(G, lvl1)
    all_nodes = list(descendants) + [lvl1]
    # enlever doublons et concaténer
    text = " ".join(sorted(set(all_nodes)))
    level1_texts[lvl1] = text

df_level1 = pd.DataFrame.from_dict(level1_texts, orient='index', columns=['concatenated_text'])
df_level1.reset_index(inplace=True)
df_level1 = df_level1.rename(columns={'index': 'level_1_category'})

display(df_level1.head())

Level 1 categories: ['Goods', 'toys & games', 'mature', 'cameras & optics', 'Real Estate', 'health & beauty', 'religious & ceremonial', 'Communication', 'home & garden', 'food, beverages & tobacco', 'sporting goods', 'Airlines', 'Ground/Cruises/Packages', 'animals & pet supplies', 'Gaming/Gambling', 'Hotels/Resorts', 'business & industrial', 'Finance Services', 'luggage & bags', 'Employment', 'office supplies', 'Travel', 'Car Rental', 'vehicles & parts', 'apparel & accessories', 'electronics', 'Services', 'furniture', 'software', 'baby & toddler', 'arts & entertainment', 'hardware', 'media', 'Dating']


,level_1_category,concatenated_text
0,Goods,Goods
1,toys & games,action & toy figures activity toys air & water...
2,mature,ammunition ammunition cases & holders ammuniti...
3,cameras & optics,binocular & monocular accessories binoculars b...
4,Real Estate,Real Estate


In [53]:
# Extract level 1 category counts

df_categories_count = pd.read_csv(count_of_products_per_level1_path)
df_categories_count

# Keep only level 1 categories present in both dataframes
common_level1 = set(df_level1['level_1_category']).intersection(set(df_categories_count['level_1_name']))
df_level1_clean = df_level1[df_level1['level_1_category'].isin(common_level1)]

print(f"Drop {len(df_level1['level_1_category'].unique()) - len(common_level1)} level 1 categories not in count_of_products_per_level1.csv")

Drop 13 level 1 categories not in count_of_products_per_level1.csv


In [4]:
print("Number of products : ", df_categories_count["count"].sum())

Number of products :  128253


## Read catalog files

In [5]:
df_catalog = pd.read_parquet(data_path, engine='pyarrow')

## Preprocessing df_catalog

In [37]:
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder

def preprocess_for_nlp(df, text_cols=['title', 'description'], brand_col='brand', id_col='hashed_external_id' , price_col = 'sale_price'):
    """
    Preprocess products DataFrame for NLP tasks:
    - Concatenate text columns into a single 'text' column
    - Encode brand as integer
    - Clean text: lowercasing, remove punctuation, multiple spaces
    - Keep hashed_external_id for final output

    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe
    text_cols : list of str
        Columns to concatenate for text
    brand_col : str
        Column to encode as integer
    id_col : str
        Column to keep for external mapping
    price_col : str
        Column for price information

    Returns
    -------
    df_preprocessed : pd.DataFrame
        DataFrame with columns: 'hashed_external_id', 'text', brand_col (encoded)
    label_enc : LabelEncoder
        Fitted LabelEncoder for the brand column
    """
    df = df.copy()

    # Fill missing values for text columns
    for col in text_cols:
        df[col] = df[col].fillna('')

    # Concatenate text columns
    df['text'] = df[text_cols].agg(' '.join, axis=1)

    # Clean text
    def clean_text(s):
        s = s.lower()
        s = re.sub(r'\s+', ' ', s)      # multiple spaces -> single space
        s = re.sub(r'[^\w\s]', '', s)   # remove punctuation
        return s.strip()

    df['text'] = df['text'].apply(clean_text)

    # Encode brand as integer
    if brand_col in df.columns:
        df[brand_col] = df[brand_col].fillna('Unknown')
        label_enc = LabelEncoder()
        df[brand_col + '_encoded'] = label_enc.fit_transform(df[brand_col])
    else:
        label_enc = None

    # Fill na for price column
    if price_col in df.columns:
        df[price_col] = df[price_col].astype(float)
        df[price_col] = df[price_col].fillna(df[price_col].median())

    # Keep hashed_external_id
    columns_to_keep = [id_col, 'text' , price_col]
    if label_enc:
        columns_to_keep.append(brand_col + '_encoded')

    return df[columns_to_keep], label_enc

# --------- Usage ---------
df_nlp, brand_encoder = preprocess_for_nlp(
    df_catalog,
    text_cols=['title','description','brand'],
    brand_col='brand',
    id_col='hashed_external_id',
    price_col = 'sale_price'
)

df_nlp.head()

,hashed_external_id,text,sale_price,brand_encoded
0,-2772291400701920348,the hoodoo tarot a divination deck and guidebo...,35.00,0
1,-4184851053829790189,disney villains tarot deck and guidebook movie...,24.99,0
2,-8778697834751578524,easy tarot created especially for beginners th...,19.95,0
3,-3541475158234224984,the proudest blue a story of hijab and family ...,17.99,0
4,-2529310467283008815,the crystal magic tarot understand and control...,24.95,0


### check language distribution

In [16]:
# Pour rendre les résultats reproductibles
DetectorFactory.seed = 0

def detect_language(text):
    """
    Detect the language of a given text.
    
    Parameters
    ----------
    text : str
        Input text
    
    Returns
    -------
    lang_code : str
        ISO 639-1 language code (e.g., 'en' for English)
    """
    try:
        return detect(text)
    except:
        return "unknown"

language_df_level1_clean = df_level1_clean["concatenated_text"].apply(detect_language)
language_df_nlp = df_nlp["text"].apply(detect_language)

In [17]:
language_df_level1_clean.value_counts()
language_df_nlp.value_counts()

text
en    128107
es        37
fr        30
it        20
ca        16
no        10
af         8
nl         7
da         5
ro         4
tl         2
sv         2
lt         2
cy         2
et         1
Name: count, dtype: int64

the vast majority of the text seems to be in English, with some other languages mixed in.

Decide to drop non-English entries for simplicity.


In [ ]:
language_df_level1_clean.value_counts().to_csv("data/language_distribution_level1_clean_count.csv", index=True)
language_df_nlp.value_counts().to_csv("data/language_distribution_nlp_count.csv", index=True)

language_df_level1_clean.to_csv("data/language_distribution_level1_clean.csv",index=True,columns=["idx","language"])
language_df_nlp.to_csv("data/language_df_nlp.csv" , index=True , columns=["idx","language"])

In [7]:
## drop non-english entries

language_df_nlp = pd.read_csv("data/language_df_nlp.csv")
df_nlp = df_nlp[language_df_nlp["language"] == 'en']

# Generate embeddings with sentence transformers

In [ ]:
# ---------- 1. Choisir le device ----------
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print("Using device:", device)

# ---------- 2. Charger le modèle ----------
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

# ---------- 3. Préparer les textes ----------
texts_catalog = df_nlp['text'].tolist()
texts_taxonomy = [
    lvl1 + " " + text
    for lvl1, text in zip(df_level1_clean['level_1_category'], df_level1_clean['concatenated_text'])
]

# ---------- 4. Encode avec batch et tqdm ----------
def encode_texts(texts, batch_size=64):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i+batch_size]
        batch_emb = model.encode(
            batch_texts,
            convert_to_numpy=True,
            device=device,
            show_progress_bar=False
        )
        embeddings.append(batch_emb)
    return np.vstack(embeddings)

emb_catalog = encode_texts(texts_catalog, batch_size=64)
emb_taxonomy = encode_texts(texts_taxonomy, batch_size=64)

print("Catalogue embeddings shape:", emb_catalog.shape)
print("Taxonomy embeddings shape:", emb_taxonomy.shape)

# ---------- 5. Sauvegarde compressée ----------
np.savez_compressed("data/emb_catalog.npz", emb_catalog)
np.savez_compressed("data/emb_taxonomy.npz", emb_taxonomy)

Using device: mps


100%|██████████| 1/1 [00:00<00:00,  1.32it/s]

Taxonomy embeddings shape: (21, 384)


## Save embeddings 

In [67]:
emb_taxonomy

array([[-0.00704797, -0.08531265,  0.02309315, ...,  0.02168325,
        -0.01170662,  0.0326249 ],
       [-0.00548955, -0.0460981 ,  0.01709225, ..., -0.07944578,
         0.00175334, -0.02320477],
       [-0.02566138, -0.05334219, -0.03936853, ..., -0.00694489,
        -0.12886402, -0.0103204 ],
       ...,
       [-0.01687125, -0.02210648,  0.00226213, ...,  0.03673438,
        -0.03365464, -0.05304608],
       [-0.07158102, -0.07273172,  0.01097328, ...,  0.04238746,
        -0.05822933,  0.00168229],
       [-0.06057917, -0.03957532,  0.04482635, ...,  0.00086623,
        -0.07034633,  0.0659965 ]], shape=(21, 384), dtype=float32)

In [68]:
# Embedding without filtering the non-english language product 

emb_catalog = np.load("data/emb_catalog.npz")["arr_0"]
emb_taxonomy = np.load("data/emb_taxonomy.npz")["arr_0"]

In [ ]:
# Embedding with filtering the non-english language product 


# emb_catalog = np.load("data/embeddings_catalog.npy")
# emb_taxonomy = np.load("data/embeddings_taxonomy.npy")

In [35]:
df_nlp_copy = df_nlp.copy()
df_nlp_copy = df_nlp_copy[df_nlp_copy["brand_encoded"]!=0]

value_count_brand = df_nlp_copy["brand_encoded"].value_counts()

value_count_brand = value_count_brand[value_count_brand > 100]

print(f"There is {len(value_count_brand)} brand that sell more than 100 product")


There is 214 brand that sell more than 100 product


there is brand that sell multiple products, so we will keep brand information for each product entry.

### KNN

In [70]:
# --------------------------
# 1. Scale price + brand
# --------------------------
price_scaler = MinMaxScaler()
price_scaled = price_scaler.fit_transform(df_nlp[['sale_price']])

brand_scaler = StandardScaler()
brand_scaled = brand_scaler.fit_transform(df_nlp[['brand_encoded']])

# --------------------------
# 2. Combine reduced embedding + numeric features
# --------------------------
X_products = np.hstack([emb_catalog, price_scaled, brand_scaled])

zeros_tax = np.zeros((emb_taxonomy.shape[0], 2))
X_tax = np.hstack([emb_taxonomy, zeros_tax])

# --------------------------
# 3. Compute similarity matrix between catalogue and taxonomy embedings
# --------------------------

similarity_matrix = cosine_similarity(X_products, X_tax)

# --------------------------
# 4. Linear assignation algorithm
# --------------------------

In [75]:
df_categories_count

,level_1_name,count
0,apparel & accessories,20000
1,home & garden,20000
2,furniture,20000
3,health & beauty,20000
4,toys & games,9879
5,electronics,8094
6,sporting goods,4674
7,arts & entertainment,4318
8,luggage & bags,4069
9,media,3708


In [76]:
# ==============================================================================
# 1. CONFIGURATION & VÉRIFICATIONS
# ==============================================================================

# Adapter selon les noms réels de tes colonnes dans df_categories_count
COL_NAME_COUNT = 'count'         # Colonne contenant le nombre (quota)
COL_NAME_CATEGORY = 'level_1_name' # Colonne contenant le nom de la catégorie

# Vérification des dimensions
n_products, n_cols_sim = similarity_matrix.shape
n_cats_df = len(df_categories_count)

if n_cols_sim != n_cats_df:
    raise ValueError(f"Erreur : La matrice a {n_cols_sim} colonnes mais le DF a {n_cats_df} catégories.")

# Extraction des quotas sous forme de tableau numpy
quotas = df_categories_count[COL_NAME_COUNT].values.astype(int)

# Vérification de la somme (doit être égale au nombre de produits)
diff_quota = n_products - quotas.sum()
if diff_quota != 0:
    print(f"⚠️ ATTENTION : La somme des quotas ({quotas.sum()}) diffère du nombre de produits ({n_products}).")
    print(f"Correction automatique : Ajout/Retrait de {diff_quota} au quota de la dernière catégorie pour équilibrer.")
    quotas[-1] += diff_quota

# ==============================================================================
# 2. CONSTRUCTION DU GRAPHE (Min-Cost Max-Flow)
# ==============================================================================

smcf = min_cost_flow.SimpleMinCostFlow()

# Indices des nœuds
SOURCE = 0
SINK = n_products + n_cats_df + 1
START_PROD_NODES = 1
START_CAT_NODES = n_products + 1

# Paramètres d'optimisation
# Ne pas demander plus de candidats TOP_K que le nombre réel de catégories.
TOP_K = min(10, n_cats_df)  # On garde au maximum 100, ou moins si moins de catégories
COST_MULTIPLIER = 100000     # Pour convertir les distances float en int

print(f"Construction du graphe pour {n_products} produits et {n_cats_df} catégories...")

# --- A. Arcs Source -> Produits ---
# Tous les produits viennent de la source (Capacité 1, Coût 0)
# Pour optimiser la boucle Python, on utilise des listes
sources = [SOURCE] * n_products
targets = list(range(START_PROD_NODES, START_PROD_NODES + n_products))
capacities = [1] * n_products
costs = [0] * n_products
smcf.add_arcs_with_capacity_and_unit_cost(sources, targets, capacities, costs)

# --- B. Arcs Produits -> Catégories (Optimisation Top-K) ---
# On identifie les K meilleures catégories pour chaque produit pour éviter de créer 4 millions d'arcs
print("Calcul des Top-K candidats...")
# argpartition met les K plus grands indices à la fin de chaque ligne
top_k_indices = np.argpartition(similarity_matrix, -TOP_K, axis=1)[:, -TOP_K:]

print("Création des arcs de coûts...")
# On prépare les listes pour le chargement en masse (batch)
u_list, v_list, cap_list, cost_list = [], [], [], []

for i in range(n_products):
    candidates = top_k_indices[i]
    row_scores = similarity_matrix[i] # Accès plus rapide
    
    for cat_idx in candidates:
        score = row_scores[cat_idx]
        
        # Coût = (1 - Similarité) converti en entier
        # On veut minimiser le coût, donc maximiser la similarité
        int_cost = int((1.0 - score) * COST_MULTIPLIER)
        
        u_list.append(START_PROD_NODES + i)
        v_list.append(START_CAT_NODES + int(cat_idx))
        cap_list.append(1)
        cost_list.append(int_cost)

smcf.add_arcs_with_capacity_and_unit_cost(u_list, v_list, cap_list, cost_list)

# --- C. Arcs Catégories -> Puits ---
# Chaque catégorie a une capacité égale à son QUOTA
u_sink, v_sink, cap_sink, cost_sink = [], [], [], []

for j, quota in enumerate(quotas):
    u_sink.append(START_CAT_NODES + j)
    v_sink.append(SINK)
    cap_sink.append(int(quota))
    cost_sink.append(0)

smcf.add_arcs_with_capacity_and_unit_cost(u_sink, v_sink, cap_sink, cost_sink)

# --- D. Définition du Flux ---
smcf.set_node_supply(SOURCE, int(n_products))
smcf.set_node_supply(SINK, -int(n_products))

# ==============================================================================
# 3. RÉSOLUTION
# ==============================================================================
print("Lancement du solveur OR-Tools...")
status = smcf.solve()

if status == smcf.OPTIMAL:
    print(f"✅ Solution Optimale trouvée ! Coût total : {smcf.optimal_cost()}")
    
    # Récupération des résultats
    final_assignments = np.zeros(n_products, dtype=int)
    
    # On parcourt les arcs pour voir où le flux est passé
    # Note : C'est plus rapide de parcourir les arcs internes que tous les arcs
    for i in range(smcf.num_arcs()):
        if smcf.flow(i) > 0:
            u = smcf.tail(i)
            v = smcf.head(i)
            # On cherche uniquement les arcs Produit -> Catégorie
            if u >= START_PROD_NODES and u < START_CAT_NODES:
                p_idx = u - START_PROD_NODES
                c_idx = v - START_CAT_NODES
                final_assignments[p_idx] = c_idx

    # Traduction Index -> Nom de catégorie
    # On assume que l'index 0 de quotas correspond à l'index 0 de df_categories_count
    cat_names_map = df_categories_count[COL_NAME_CATEGORY].values
    predicted_labels = [cat_names_map[i] for i in final_assignments]
    
    # Ajout au DataFrame original (si tu l'as sous la main)
    # df_nlp['final_category'] = predicted_labels
    
    print("Exemple des 5 premières assignations :", predicted_labels[:5])
    
else:
    print("❌ Pas de solution trouvée. Vérifiez que la somme des quotas est exacte et que Top-K est suffisant.")

Construction du graphe pour 128253 produits et 21 catégories...
Calcul des Top-K candidats...
Création des arcs de coûts...
Lancement du solveur OR-Tools...
✅ Solution Optimale trouvée ! Coût total : 10359029364
Exemple des 5 premières assignations : ['toys & games', 'apparel & accessories', 'toys & games', 'toys & games', 'toys & games']


In [85]:
df_result = pd.DataFrame(data= predicted_labels , columns=["predicted_level_1_cat"])
df_result["hashed_external_id"] = df_nlp["hashed_external_id"]
df_result.to_csv("data/predicted_level_1.csv")